**Desafío 1** : Construir un modelo de inteligencia artificial capaz de clasificar noticias de prensa según su categoría temática principal. 

category: sociedad, salud, politica, medioambiente, internacional, entretenimiento, economia, deportes, cultura, cienciatecnologia

In [1]:
import pandas as pd

# Lee el archivo CSV en un DataFrame de pandas
#df = pd.read_csv("train_data.csv",thousands=',')
df = pd.read_csv("final_output.csv",thousands=',')

df

,title,text,target
0,"Presentan la Estrategia Regional de Ciencia, T...",En el Distrito de Innovación V21 en Viña del ...,cienciatecnologia
1,"Ciencia, tecnología, conocimiento e innovación...","miércoles, 14 junio, 2023 a las 21:29 Editor A...",cienciatecnologia
2,Región de Coquimbo contará con el primer parqu...,El Polo Científico y Tecnológico de la Univers...,cienciatecnologia
3,Armada invita a investigadores a realizar cien...,"Fortalecer el sistema de Ciencia, Tecnología, ...",cienciatecnologia
4,Silvia Díaz es designada para dirigir el Conse...,"El Presidente de la República, Gabriel Boric, ...",cienciatecnologia
...,...,...,...
1995,Día del Niño 2023 en Chile: ¿cuánto falta?,TENDENCIAS.- En el mes de agosto en Chile se s...,sociedad
1996,[OPINION] Rompiendo el micromachismo cultural ...,El reciente caso de apología a la violencia de...,sociedad
1997,135 años al servicio de la educación de Osorno...,Fundado con el nombre de Liceo de Hombres de O...,sociedad
1998,Presidente Boric promulga Ley TEA,"La ley establece la inclusión, atención integr...",sociedad


In [2]:
#df['clase'] = df['clase'].replace(['ciencia', 'tecnologia'], 'cienciatecnologia')
#df['clase'] = df['clase'].replace('medio_ambiente', 'medioambiente')
#df['clase'] = df['clase'].replace('accidentes', 'sociedad')
#df['clase'] = df['clase'].replace('educacion', 'cultura')

#df

In [4]:
import random

# Obtén 1000 filas aleatorias del DataFrame
random_rows = df.sample(n=1000, random_state=42)

# Encuentra las clases únicas en la columna "clase"
unique_classes = random_rows["target"].unique()

unique_classes

array(['sociedad', 'cultura', 'medioambiente', 'entretenimiento', 'salud',
       'cienciatecnologia', 'internacional', 'deportes', 'economia',
       'politica'], dtype=object)

In [5]:
## Crea la lista de tuplas con las etiquetas de clases
data_list = []
for _, row in random_rows.iterrows():
    title = row["title"]
    clase = row["target"]
    cats = {cls: 1 if cls == clase else 0 for cls in unique_classes}
    data_list.append((title, {"cats": cats}))

In [7]:
#Ejecutar dos veces
import spacy
import random
from spacy.training.example import Example
from spacy.util import minibatch

from spacy.pipeline.tok2vec import DEFAULT_TOK2VEC_MODEL

# Datos de entrenamiento
TRAIN_DATA = data_list

# Inicializar spaCy con el modelo de lenguaje es_core_news_sm
nlp = spacy.load("es_core_news_sm")

train_examples = []

for example in TRAIN_DATA:
    train_examples.append(Example.from_dict(nlp.make_doc(example[0]), example[1]))

def get_examples():
    return train_examples

In [8]:
model = {
            "@architectures": "spacy.TextCatCNN.v2",
            "exclusive_classes": True,
            "tok2vec": DEFAULT_TOK2VEC_MODEL,
        }

# Crear un componente TextCategorizer con un modelo CNN
textcat = nlp.add_pipe("textcat", config={"model": model})



In [9]:
textcat.initialize(get_examples)

In [10]:
from spacy.util import compounding

# Entrenar el modelo
with nlp.select_pipes(enable="textcat"):
    optimizer = nlp.begin_training()
    for epoch in range(100):
        losses = {}
        random.shuffle(TRAIN_DATA)
        # Dividir los datos en lotes y actualizar el modelo
        for batch in minibatch(TRAIN_DATA, size=compounding(2.0, 32.0, 1.001)):
            texts, annotations = zip(*batch)
            example = []
            # Actualizar el modelo con iteraciones
            for i in range(len(texts)):
                doc = nlp.make_doc(texts[i])
                example.append(Example.from_dict(doc, annotations[i]))
            nlp.update(example, drop=0.5, losses=losses)
        print(losses)

{'textcat': 42.04787262529135}
{'textcat': 39.407292203977704}
{'textcat': 34.1174188349396}
{'textcat': 26.65628725447459}
{'textcat': 19.59091913700999}
{'textcat': 14.185280643296622}
{'textcat': 10.900772118390027}
{'textcat': 6.88285094215385}
{'textcat': 6.937038271598017}
{'textcat': 5.522471897606755}
{'textcat': 3.9843649150106972}
{'textcat': 3.6106549937896335}
{'textcat': 3.351309838361964}
{'textcat': 2.4135422545970315}
{'textcat': 2.789866827136741}
{'textcat': 2.753406449486765}
{'textcat': 2.6314961797474847}
{'textcat': 1.9298980198088016}
{'textcat': 2.2654393788621503}
{'textcat': 2.9632438219715516}
{'textcat': 2.517671748243197}
{'textcat': 0.9807545487040265}
{'textcat': 1.1602422296532973}
{'textcat': 1.7863832779078959}
{'textcat': 1.5120839585148365}
{'textcat': 0.983756489226982}
{'textcat': 1.8703214652866966}
{'textcat': 1.3476757326938766}
{'textcat': 1.447258636445983}
{'textcat': 1.3156574518309874}
{'textcat': 1.4332365998508507}
{'textcat': 1.539878447

In [11]:
# Guardar el modelo entrenado en disco
nlp.to_disk("modeloTarea1_1_CNNv2")

In [12]:
# Cargar el modelo entrenado
nlp_loaded = spacy.load("modeloTarea1_1_CNNv2")#modelo_clasificador_noticias")






***Desafío 2***: Geolocalizar eventos en noticias

Utilizando técnicas de tratamiento automático del lenguaje, el objetivo del desafío es estructurar la información de una noticia de la manera siguiente:

event: principal evento descrito en la noticia
category: categoria temática del evento
address: dirección dónde occurió el evento en formato: calle número, comuna, país
latitud: latitud del evento
longitud: longitud del evento




Resultado: El formato del archivo CSV output que entregar contiene las columnas siguientes: id_news, event, category, address, latitud, longitud


Evaluación del desafío: Se evaluará su método en base a 100 noticias etiquetadas con las métricas siguientes:
Category: Precision y Recall
Longitud y Latitud: Exactitud (con una margen de error de 100 metros)


In [13]:
from transformers import AutoModelForQuestionAnswering, AutoTokenizer
from transformers import pipeline
from geopy.geocoders import Nominatim

geolocator = Nominatim(user_agent="Sophia")

modelAdd="mrm8488/bert-base-spanish-wwm-cased-finetuned-spa-squad2-es"

tokenizer = AutoTokenizer.from_pretrained(modelAdd)

modelAdd = AutoModelForQuestionAnswering.from_pretrained(modelAdd)

nlp2 = pipeline("question-answering", model=modelAdd, tokenizer=tokenizer)

C:\Users\sebaa\INFO\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of the model checkpoint at mrm8488/bert-base-spanish-wwm-cased-finetuned-spa-squad2-es were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing

In [14]:
df2 = pd.read_csv('dataset_agosto2024.csv',thousands=',')
df2 = df2.dropna()
df2 = df2[df2['text'] != '(empty)']

questions = [
    "¿Cuál es el evento principal descrito en el texto?",
    "¿En que país tuvo lugar el hecho?",
    "¿En que Comuna ocurrio el hecho?",
    "¿En que calle ocurrio el hecho?"
]

df2['event'] = ''
df2['category'] = ''
df2['address'] = ''
df2['latitud'] = ''
df2['longitud'] = ''

df2 = df2.sample(n=100)

In [15]:
for index, row in df2.iterrows():  
    Answer = []
    for question in questions: 
        result = nlp2(question=question, tokenizer=tokenizer, model=modelAdd, context=row['text'])
        Answer.append(result['answer'])
        #print(question)
        #print(result['answer'])
    
    #Event
    df2.at[index,'event'] = Answer[0]
    
    #Address
    lugar = Answer[2] +", "+ Answer[1]
    df2.at[index,'address'] = lugar
    
    #Latitude, Longitude
    try:
        location = geolocator.geocode(lugar)
        #print(location.address)
        #print((location.latitude, location.longitude))
        df2.at[index,'latitud'] = location.latitude
        df2.at[index,'longitud'] = location.longitude
        #print(row['address'])
    except Exception as e:
        print("Error con la locacion, colocando coordenadas predetemiandas")
        df2.at[index,'latitud'] = 0.0
        df2.at[index,'longitud'] = 0.0        
    
    #Categoria
    doc = nlp_loaded(row['title'])
    #print("Categorías predichas:")
    maxscore = 0.0
    maxlabel = ''
    for label, score in doc.cats.items():
        if (score > maxscore):
            maxscore = score
            maxlabel = label
    #print(f"{maxlabel}: {maxscore:.4f}")
    df2.at[index,'category'] = maxlabel

output_columns = ['id_news', 'event', 'category', 'address', 'latitud', 'longitud']
df2[output_columns].to_csv('desafio2_output.csv', index=False)
print("Creado archivo .csv")

Error con la locacion, colocando coordenadas predetemiandas
Error con la locacion, colocando coordenadas predetemiandas
Error con la locacion, colocando coordenadas predetemiandas
Error con la locacion, colocando coordenadas predetemiandas
Error con la locacion, colocando coordenadas predetemiandas
Error con la locacion, colocando coordenadas predetemiandas
Error con la locacion, colocando coordenadas predetemiandas
Error con la locacion, colocando coordenadas predetemiandas
Error con la locacion, colocando coordenadas predetemiandas
Error con la locacion, colocando coordenadas predetemiandas
Error con la locacion, colocando coordenadas predetemiandas
Error con la locacion, colocando coordenadas predetemiandas
Error con la locacion, colocando coordenadas predetemiandas
Error con la locacion, colocando coordenadas predetemiandas
Error con la locacion, colocando coordenadas predetemiandas
Error con la locacion, colocando coordenadas predetemiandas
Error con la locacion, colocando coorden